In [126]:
from iqoptionapi.stable_api import IQ_Option
import logging, sys, time, configparser
from talib.abstract import ATR, EMA
import numpy as np
import pandas as pd
from datetime import date
from csv import writer

def login(verbose = False, iq = None, checkConnection = False):
    
    if verbose:
        logging.basicConfig(level=logging.DEBUG,format='%(asctime)s %(message)s')

    if iq == None:
      print("Trying to connect to IqOption")
      iq=IQ_Option('@gmail.com','Pass') # YOU HAVE TO ADD YOUR USERNAME AND PASSWORD
      iq.connect()

    if iq != None:
      while True:
        if iq.check_connect() == False:
          print('Error when trying to connect')
          print(iq)
          print("Retrying")
          iq.connect()
        else:
          if not checkConnection:
            print('Successfully Connected!')
          break
        time.sleep(3)

    iq.change_balance("PRACTICE") #or real
    return iq

def higher(iq,Money,Actives):
    
    done,id = iq.buy(Money,Actives,"call",1)
    print(Money, id)
    if not done:
        print('Error call')
        print(done, id)
        exit(0)
    
    return id


def lower(iq,amount,pair):
    
    done,id = iq.buy(amount,pair,"put",1)
    print(amount, id)
    
    if not done:
        print('Error put')
        print(done, id)
        exit(0)
    
    return id

def atr(values, period):
    return ATR(values['high'], values['low'], values['close'],period)

def ema(values, period, offset):
    ema = EMA(values['close'], timeperiod=period)
    return ema
def upper(ema):
    upperb = ema + offset * atr(values, period)
    return upper
def lower(ema):
    lowerb = ema - offset * atr(values, period)
    return lowerb
def keltner(values, period, offset):
    ema = EMA(values['close'], timeperiod=period)
    upperb = ema + offset * atr(values, period)
    lowerb = ema - offset * atr(values, period)
    return upperb, lowerb, ema

def get_candles(pair, candle_freq):
    candles = API.get_realtime_candles(pair, candle_freq)
    values = {'open': np.array([]), 
                  'high': np.array([]), 
                  'low': np.array([]), 
                  'close': np.array([]), 
                  'volume': np.array([]) }
        
        
    for x in candles:
        values['open'] = np.append(values['open'], candles[x]['open'])
        values['high'] = np.append(values['high'], candles[x]['max'])
        values['low'] = np.append(values['low'], candles[x]['min'])
        values['close'] = np.append(values['close'], candles[x]['close'])
        values['volume'] = np.append(values['volume'], candles[x]['volume'])

    df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in values.items() ]))
    return df, values

logging.disable(level=(logging.DEBUG))


API = IQ_Option("@gmail.com", 'Pass')
API.connect()

API.change_balance('PRACTICE') # PRACTICE / REAL

if API.check_connect():
    print('\n\nSuccessful connection')
else:
    print('\n Error connecting')
    sys.exit()


pair = 'EURUSD-OTC'
amount = 10

period_EMA2 = 25
candle_timeframe = 60
num_candles_to_save = 3000
API.start_candles_stream(pair, candle_timeframe, num_candles_to_save)

if_mid_way = None
if_upperb = None

GupperB = False
LlowerB = False

trade = False

df, values = get_candles(pair, candle_timeframe) 
df.drop(columns = ["volume"])



Successful connection


,open,high,low,close
0,0.921719,0.921768,0.921699,0.921707
1,0.921698,0.921797,0.921697,0.921782
2,0.921809,0.921938,0.921807,0.921938
3,0.921934,0.921936,0.921835,0.921853
4,0.921857,0.921927,0.921849,0.921877
...,...,...,...,...
1222,0.921092,0.921118,0.921064,0.921095
1223,0.921096,0.921112,0.921031,0.921066
1224,0.921101,0.921209,0.921065,0.921198
1225,0.921166,0.921198,0.921044,0.921082


In [127]:

period_EMA2 = 25
candle_timeframe = 60
num_candles_to_save = 3000
API.start_candles_stream(pair, candle_timeframe, num_candles_to_save)

if_mid_way = None
if_upperb = None

GupperB = False
LlowerB = False

trade = False



df_temp = pd.read_csv("C:/Users/HP/Desktop/Binary_bots/Untitled Folder/EURUSD1.csv")
df_temp = df_temp.drop(columns = ["volume"])
df_temp

,date,time,open,high,low,close
0,2021.10.26,4:52,1.16062,1.16070,1.16061,1.16067
1,2021.10.26,4:53,1.16067,1.16069,1.16048,1.16049
2,2021.10.26,4:54,1.16050,1.16055,1.16047,1.16048
3,2021.10.26,4:55,1.16048,1.16050,1.16047,1.16048
4,2021.10.26,4:56,1.16050,1.16052,1.16037,1.16038
...,...,...,...,...,...,...
64996,2021.12.31,9:08,1.13156,1.13156,1.13153,1.13153
64997,2021.12.31,9:09,1.13154,1.13154,1.13151,1.13153
64998,2021.12.31,9:10,1.13153,1.13153,1.13134,1.13135
64999,2021.12.31,9:11,1.13135,1.13138,1.13135,1.13138


In [128]:
upperb, lowerb, ema = keltner(df_temp, 25, 2)
upb = pd.DataFrame(pd.Series(upperb), columns = ["upperb"])
ema = pd.DataFrame(pd.Series(ema), columns = ["ema"])
lwb = pd.DataFrame(pd.Series(lowerb), columns = ["lowerb"])
df2 = pd.concat([df_temp, upb], axis=1)
df2 = pd.concat([df2, ema], axis=1)
df2 = pd.concat([df2, lwb], axis=1)
# df2 = df2.drop(columns = ["volume"])
                 
df2 = df2.dropna().reset_index(drop=True)
df2

,date,time,open,high,low,close,upperb,ema,lowerb
0,2021.10.26,5:17,1.16066,1.16069,1.16057,1.16060,1.160647,1.160489,1.160330
1,2021.10.26,5:18,1.16060,1.16060,1.16045,1.16046,1.160651,1.160487,1.160323
2,2021.10.26,5:19,1.16046,1.16046,1.16036,1.16037,1.160643,1.160478,1.160312
3,2021.10.26,5:20,1.16037,1.16037,1.16026,1.16026,1.160629,1.160461,1.160293
4,2021.10.26,5:21,1.16026,1.16027,1.16020,1.16024,1.160611,1.160444,1.160277
...,...,...,...,...,...,...,...,...,...
64971,2021.12.31,9:08,1.13156,1.13156,1.13153,1.13153,1.131468,1.131335,1.131203
64972,2021.12.31,9:09,1.13154,1.13154,1.13151,1.13153,1.131480,1.131350,1.131221
64973,2021.12.31,9:10,1.13153,1.13153,1.13134,1.13135,1.131490,1.131350,1.131211
64974,2021.12.31,9:11,1.13135,1.13138,1.13135,1.13138,1.131489,1.131353,1.131216


In [131]:
#######################################
from google.oauth2 import service_account
from googleapiclient.discovery import build
from tqdm import tqdm
#Google Sheets API
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'C:/Users/HP/Desktop/Binary_bots/binary-bot-master/key.json'

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

ID = '1ctkjlfOvR75lp2O6SwfYU1CkLpW6TWP43EvjRtO3deM'
service = build('sheets', 'v4', credentials=creds)

# request = service.spreadsheets().values().clear(spreadsheetId=ID, range="BackTest!A1", body=clear_values_request_body)
#         result = service.spreadsheets().values().append(
#                 spreadsheetId=ID, range="BackTest!A1:E1",
#                 valueInputOption="USER_ENTERED", insertDataOption="INSERT_ROWS", body = {"values":data}).execute()
    
rangeAll = '{0}!A2:Z'.format( "BackTest" )
body = {}
resultClear = service.spreadsheets( ).values( ).clear( spreadsheetId=ID, range=rangeAll,
                                                       body=body ).execute( )
#######################################



if_mid_way = None
if_upperb = None
GupperB = False
LlowerB = False
trade = False
higher = None
lower = None
martingale = 0

df_to_append = pd.DataFrame(columns = ["Date", "Time","Outcome", "Balance"])


balance = 10000
for i in tqdm(range(0, df2.shape[0]-1)):
    date = df2['date'][i]
    time = df2['time'][i]
    close = df2['close'][i]
    op = df2['open'][i]
    upperb = df2['upperb'][i]
    ema = df2['ema'][i]
    lowerb = df2['lowerb'][i]
    
    
    #if red candle
    if GupperB == True and if_mid_way == True:
        if close < op and close <= upperb:
            if_mid_way = None
            higher = True
            trade = True
            GupperB = False

    #If green candle
    if LlowerB == True and if_mid_way == True:
        if close > op and close >= lowerb:
            if_mid_way = None
            lower = True
            trade = True
            LlowerB = False

    #If greater than upperbound and touched midway
    if if_mid_way == True and if_upperb == True:
        if close > upperb:
            GupperB = True

    #If less than lowerbound and touched midway
    elif if_mid_way == True and if_upperb == False:
        if close < lowerb:
            LlowerB = True

    #If crosses the midway 
    if if_upperb == True:
        if close < ema:
            if_mid_way = True
    elif if_upperb == False:
        if close>ema:
            if_mid_way = True

    #If in either top or bottom bound
    if close > ema:
        if_upperb = True
    elif close < ema:
        if_upperb = False
        
    if trade == True:
#         print('...')
        close_next = df2['close'][i+1]
        op_next = df2['open'][i+1]
        if higher == True:
            if close_next<op_next:
                balance = balance + 10
                data = [date, time, "win", balance]
            elif close_next==op_next:
                data = [date, time, "equal", balance]
            else:
                balance = balance - 10
                data = [date, time, "lose", balance]
        elif lower == True:
            if close_next>op_next:
                balance = balance + 10
                data = [date, time, "win", balance]
            elif close_next==op_next:
                data = [date, time, "equal", balance]
            else:
                balance = balance - 10
                data = [date, time, "lose", balance]
            
        higher = None
        lower = None
        
        a_series = pd.Series(data, index = df_to_append.columns)
        df_to_append = df_to_append.append(a_series, ignore_index=True)
#         print(df_to_append)
        

        trade = False
df_to_append.to_csv('out.csv',index=False)

100%|██████████| 64975/64975 [00:04<00:00, 13832.44it/s]


In [132]:
df_to_append

,Date,Time,Outcome,Balance
0,2021.10.26,5:35,lose,9990
1,2021.10.26,6:20,lose,9980
2,2021.10.26,6:34,lose,9970
3,2021.10.26,7:04,lose,9960
4,2021.10.26,7:15,lose,9950
...,...,...,...,...
1847,2021.12.31,6:30,equal,10300
1848,2021.12.31,6:50,win,10310
1849,2021.12.31,7:15,win,10320
1850,2021.12.31,8:12,lose,10310


In [133]:
df_to_append.groupby(pd.Grouper(key='Time', axis=0, 
                      freq='M')).sum()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'